In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import logging
from scipy.ndimage import zoom
from giza_actions.action import action
from giza_actions.task import task
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

2024-02-04 19:51:55.134902: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 19:51:55.158903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 19:51:55.158927: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 19:51:55.159529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 19:51:55.163348: I tensorflow/core/platform/cpu_feature_guar

In [21]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Resize for 14 * 14 images
train_images_tf_reshaped = tf.reshape(train_images_tf, [-1, 28, 28, 1])  # Reshape to [num_samples, height, width, channels]
test_images_tf_reshaped = tf.reshape(test_images_tf, [-1, 28, 28, 1])

# Downsample images
train_images_tf_downsampled = tf.image.resize(train_images_tf_reshaped, [14, 14], method='bilinear')
test_images_tf_downsampled = tf.image.resize(test_images_tf_reshaped, [14, 14], method='bilinear')

# Flatten the images back to [num_samples, 14*14]
train_images_tf_downsampled = tf.reshape(train_images_tf_downsampled, [-1, 14*14])
test_images_tf_downsampled = tf.reshape(test_images_tf_downsampled, [-1, 14*14])

2024-02-04 19:57:06.243648: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-04 19:57:06.244279: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-04 19:57:06.244341: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [28]:
from PIL import Image
img = test_images[0]
img = Image.fromarray(img)
img.save("./zero.png")

In [3]:
input_size = 196  # 14x14
hidden_size = 10 
num_classes = 10
num_epochs = 10
batch_size = 256
learning_rate = 0.001

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes) 

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [4]:
def resize_images(images):
    return np.array([zoom(image[0], (0.5, 0.5)) for image in images])

@task(name=f'Prepare Datasets')
def prepare_datasets():
    print("Prepare dataset...")
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False)

    x_train = resize_images(train_dataset)
    x_test = resize_images(test_dataset)

    x_train = torch.tensor(x_train.reshape(-1, 14*14).astype('float32') / 255)
    y_train = torch.tensor([label for _, label in train_dataset], dtype=torch.long)

    x_test = torch.tensor(x_test.reshape(-1, 14*14).astype('float32') / 255)
    y_test = torch.tensor([label for _, label in test_dataset], dtype=torch.long)

    print("✅ Datasets prepared successfully")

    return x_train, y_train, x_test, y_test

In [5]:
@task(name=f'Create Loaders')
def create_data_loaders(x_train, y_train, x_test, y_test):
    print("Create loaders...")

    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

    print("✅ Loaders created!")

    return train_loader, test_loader

In [6]:
@task(name=f'Train model')
def train_model(train_loader):
    print("Train model...")

    model = NeuralNet(input_size, hidden_size, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device).reshape(-1, 14*14)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    print("✅ Model trained successfully")
    return model

In [7]:
@task(name=f'Test model')
def test_model(model, test_loader):
    print("Test model...")
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in test_loader:
            images = images.to(device).reshape(-1, 14*14)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network on the 10000 test images: {acc} %')

In [8]:

@action(name=f'Execution', log_prints=True )
def execution():
    x_train, y_train, x_test, y_test = prepare_datasets()
    train_loader, test_loader = create_data_loaders(
        x_train, y_train, x_test, y_test)
    model = train_model(train_loader)
    test_model(model, test_loader)


execution()

19:35:52.325 | INFO    | prefect.engine - Created flow run 'amber-peacock' for flow 'Execution'

19:35:52.329 | INFO    | Flow run 'amber-peacock' - View at https://actions-server-guuu-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/c58efcf5-c51d-44e3-abb0-ed66127db018

19:35:55.564 | INFO    | Flow run 'amber-peacock' - Created task run 'Prepare Datasets-0' for task 'Prepare Datasets'

19:35:55.573 | INFO    | Flow run 'amber-peacock' - Executing 'Prepare Datasets-0' immediately...

19:35:56.993 | INFO    | Task run 'Prepare Datasets-0' - Prepare dataset...

...

19:36:21.140 | INFO    | Task run 'Test model-0' - Accuracy of the network on the 10000 test images: 92.05 %

19:36:21.514 | INFO    | Task run 'Test model-0' - Finished in state Completed()

19:36:21.515 | WARNING | Task run 'Test model-0' - Task run 'Test model-0' finished in state Completed()

19:36:21.897 | INFO    | Flow run 'amber-peacock' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NeuralNet`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

In [10]:
import torch.onnx

@task(name=f'Convert To ONNX')
def convert_to_onnx(model, onnx_file_path):
    dummy_input = torch.randn(1, input_size).to(device)
    torch.onnx.export(model, dummy_input, onnx_file_path,
                      export_params=True, opset_version=10, do_constant_folding=True)

    print(f"Model has been converted to ONNX and saved as {onnx_file_path}")


@action(name="Action: Convert To ONNX", log_prints=True)
def execution():
    x_train, y_train, x_test, y_test = prepare_datasets()
    train_loader, test_loader = create_data_loaders(
        x_train, y_train, x_test, y_test)
    model = train_model(train_loader)
    test_model(model, test_loader)

    # Convert to ONNX
    onnx_file_path = "mnist_model.onnx"
    convert_to_onnx(model, onnx_file_path)


execution()

19:39:01.477 | INFO    | prefect.engine - Created flow run 'tungsten-dragonfly' for flow 'Action: Convert To ONNX'

19:39:01.478 | INFO    | Flow run 'tungsten-dragonfly' - View at https://actions-server-guuu-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/b968f74a-e51d-40b1-90ce-b9d9070293de

19:39:04.775 | INFO    | Flow run 'tungsten-dragonfly' - Created task run 'Prepare Datasets-0' for task 'Prepare Datasets'

19:39:04.777 | INFO    | Flow run 'tungsten-dragonfly' - Executing 'Prepare Datasets-0' immediately...

19:39:06.580 | INFO    | Task run 'Prepare Datasets-0' - Prepare dataset...

...

19:39:26.088 | INFO    | Task run 'Convert To ONNX-0' - Model has been converted to ONNX and saved as mnist_model.onnx

19:39:26.474 | INFO    | Task run 'Convert To ONNX-0' - Finished in state Completed()

19:39:26.475 | WARNING | Task run 'Convert To ONNX-0' - Task run 'Convert To ONNX-0' finished in state Completed()

19:39:26.852 | INFO    | Flow run 'tungsten-dragonfly' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NeuralNet`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

In [37]:
import torch.nn.functional as F

def preprocess_image(image_path):
    from PIL import Image
    import numpy as np

    # Load image, convert to grayscale, resize and normalize
    image = Image.open(image_path).convert('L')
    # Resize to match the input size of the model
    image = image.resize((14, 14))
    image = np.array(image).astype('float32') / 255
    image = image.reshape(1, 196)  # Reshape to (1, 196) for model input
    return image

image = preprocess_image("./zero.png")

In [38]:
model = GizaModel(model_path="./mnist_model.onnx")
result = model.predict(input_feed={"onnx::Gemm_0": image}, verifiable = True,
                       output_dtype = "tensor_fixed_point")

20:13:00.807 | ERROR   | root - An error occurred in predict: 'GizaModel' object has no attribute 'uri'

In [39]:

import torch.nn.functional as F


@task(name=f'Preprocess Image')
def preprocess_image(image_path):
    from PIL import Image
    import numpy as np

    # Load image, convert to grayscale, resize and normalize
    image = Image.open(image_path).convert('L')
    # Resize to match the input size of the model
    image = image.resize((14, 14))
    image = np.array(image).astype('float32') / 255
    image = image.reshape(1, 196)  # Reshape to (1, 196) for model input
    return image


@task(name=f'Prediction with ONNX')
def prediction(image):
    model = GizaModel(model_path="./mnist_model.onnx")

    result = model.predict(
        input_feed={"onnx::Gemm_0": image}, verifiable=False
    )

    # Convert result to a PyTorch tensor
    result_tensor = torch.tensor(result)
    # Apply softmax to convert to probabilities
    probabilities = F.softmax(result_tensor, dim=1)
    # Use argmax to get the predicted class
    predicted_class = torch.argmax(probabilities, dim=1)

    return predicted_class.item()


@action(name=f'Execution: Prediction with ONNX', log_prints=True )
def execution():
    image = preprocess_image("./zero.png")
    predicted_digit = prediction(image)
    print(f"Predicted Digit: {predicted_digit}")

    return predicted_digit


execution()

/home/guy1m0/.pyenv/versions/3.11.5/lib/python3.11/site-packages/prefect/tasks.py:337: UserWarning: A task named 'Preprocess Image' and defined at '/home/guy1m0/.pyenv/versions/3.11.5/lib/python3.11/site-packages/giza_actions/task.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/home/guy1m0/.pyenv/versions/3.11.5/lib/python3.11/site-packages/prefect/tasks.py:337: UserWarning: A task named 'Prediction with ONNX' and defined at '/home/guy1m0/.pyenv/versions/3.11.5/lib/python3.11/site-packages/giza_actions/task.py:18' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/home/guy1m0/.pyenv/versions/3.11.5/lib/python3.11/site-packages/prefect/flows.py:337: UserWarning: A flow named 'Execution: Prediction with ONNX' and defined at '/home/guy1m0/.pyenv/versions/3.11.5/lib/python3.11

20:13:25.095 | INFO    | prefect.engine - Created flow run 'cerulean-alpaca' for flow 'Execution: Prediction with ONNX'

20:13:25.096 | INFO    | Flow run 'cerulean-alpaca' - View at https://actions-server-guuu-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/1a219e0d-7934-4d54-96a2-5b4844cf1332

20:13:27.290 | INFO    | Flow run 'cerulean-alpaca' - Created task run 'Preprocess Image-0' for task 'Preprocess Image'

20:13:27.301 | INFO    | Flow run 'cerulean-alpaca' - Executing 'Preprocess Image-0' immediately...

...

20:13:30.839 | INFO    | Task run 'Prediction with ONNX-0' - Finished in state Completed()

20:13:30.841 | WARNING | Task run 'Prediction with ONNX-0' - Task run 'Prediction with ONNX-0' finished in state Completed()

20:13:30.842 | INFO    | Flow run 'cerulean-alpaca' - Predicted Digit: 7

20:13:31.249 | INFO    | Flow run 'cerulean-alpaca' - Finished in state Completed()

7

In [40]:
MODEL_ID = 253  # Update with your model ID
VERSION_ID = 1  # Update with your version ID


@task(name=f'Prediction with Cairo')
def prediction(image, model_id, version_id):
    model = GizaModel(id=model_id, version=version_id)

    (result, request_id) = model.predict(
        input_feed={"image": image}, verifiable=True, output_dtype="tensor_fixed_point"
    )

    # Convert result to a PyTorch tensor
    result_tensor = torch.tensor(result)
    # Apply softmax to convert to probabilities
    probabilities = F.softmax(result_tensor, dim=1)
    # Use argmax to get the predicted class
    predicted_class = torch.argmax(probabilities, dim=1)

    return predicted_class.item(), request_id


@action(name=f'Execution: Prediction with Cairo', log_prints=True)
def execution():
    image = preprocess_image("./zero.png")
    (result, request_id) = prediction(image, MODEL_ID, VERSION_ID)
    print("Result: ", result)
    print("Request id: ", request_id)

    return result, request_id


execution()

20:14:35.112 | INFO    | prefect.engine - Created flow run 'agile-hummingbird' for flow 'Execution: Prediction with Cairo'

20:14:35.113 | INFO    | Flow run 'agile-hummingbird' - View at https://actions-server-guuu-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/fe0638f2-39ea-442f-b3a7-4f4d164cc248

20:14:36.197 | INFO    | Flow run 'agile-hummingbird' - Created task run 'Preprocess Image-0' for task 'Preprocess Image'

20:14:36.213 | INFO    | Flow run 'agile-hummingbird' - Executing 'Preprocess Image-0' immediately...

20:14:37.354 | INFO    | Task run 'Preprocess Image-0' - Finished in state Completed()

...

20:14:44.363 | WARNING | Task run 'Prediction with Cairo-0' - Task run 'Prediction with Cairo-0' finished in state Completed()

20:14:44.365 | INFO    | Flow run 'agile-hummingbird' - Result:  7

20:14:44.366 | INFO    | Flow run 'agile-hummingbird' - Request id:  "2311e3518d0640cb9b5cb75d2daf655b"

20:14:44.752 | INFO    | Flow run 'agile-hummingbird' - Finished in state Completed()

(7, '"2311e3518d0640cb9b5cb75d2daf655b"')

In [41]:
img = test_images[1]
img = Image.fromarray(img)
img.save("./zero.png")

In [42]:
import time
import psutil

def monitor_function():
    start_time = time.time()
    # Monitor memory usage at the start
    process = psutil.Process()
    start_memory = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB

    # Call your function here
    result, request_id = execution()

    # Monitor memory usage at the end
    end_memory = process.memory_info().rss / (1024 * 1024)  # Convert bytes to MB
    end_time = time.time()

    memory_used = end_memory - start_memory
    time_taken = end_time - start_time

    print(f"Memory used: {memory_used} MB")
    print(f"Execution time: {time_taken} seconds")

    return result, request_id, memory_used, time_taken

# Now call the monitor function
monitor_function()


20:25:33.878 | INFO    | prefect.engine - Created flow run 'spry-iguana' for flow 'Execution: Prediction with Cairo'

20:25:33.879 | INFO    | Flow run 'spry-iguana' - View at https://actions-server-guuu-dblzzhtf5q-ew.a.run.app/flow-runs/flow-run/43b5c8ca-e279-4bef-97f9-70d0e04b221d

20:25:34.904 | INFO    | Flow run 'spry-iguana' - Created task run 'Preprocess Image-0' for task 'Preprocess Image'

20:25:34.905 | INFO    | Flow run 'spry-iguana' - Executing 'Preprocess Image-0' immediately...

20:25:36.070 | INFO    | Task run 'Preprocess Image-0' - Finished in state Completed()

...

20:25:41.329 | INFO    | Flow run 'spry-iguana' - Result:  2

20:25:41.330 | INFO    | Flow run 'spry-iguana' - Request id:  "aa2a2d305e5441179d20234ecb9891c5"

20:25:41.699 | INFO    | Flow run 'spry-iguana' - Finished in state Completed()

Memory used: 1.875 MB
Execution time: 9.263678073883057 seconds


(2, '"aa2a2d305e5441179d20234ecb9891c5"', 1.875, 9.263678073883057)

In [43]:
import requests
from giza import API_HOST

# Update the following variables with your specific details
MODEL_ID = 253
VERSION_ID = 1
DEPLOYMENT_ID = 1
REQUEST_ID = "aa2a2d305e5441179d20234ecb9891c5"
API_KEY = 'Z8WQhrooDRbjh57-1fiuIw'

url = f'{API_HOST}/api/v1/models/{MODEL_ID}/versions/{VERSION_ID}/deployments/{DEPLOYMENT_ID}/proofs/{REQUEST_ID}:download'
headers = {"X-API-KEY": API_KEY}
d_url = requests.get(url, headers=headers).json()["download_url"]
proof = requests.get(d_url)

with open("zk.proof", "wb") as f:
     f.write(proof.content)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

/home/guy1m0/.giza/.api_key.json